In [1]:
import os
from os.path import isfile
import pandas as pd
import pickle
from compute_ale import compute_ale
from compile_studies import compile_studies
from contribution import contribution
from compute_contrast import compute_contrast
%matplotlib inline

cwd = os.getcwd()
raw_folder = cwd + '/DataRaw/'
pickle_folder = cwd + '/DataPickle/'

'''filename = askopenfilename()
df = pd.read_excel(filename, engine='openpyxl', header=None)'''

df = pd.read_excel(raw_folder + 'Metastocalculate.xlsx', engine='openpyxl', header=None)
with open(pickle_folder + 'experiments.pickle', 'rb') as f:
    experiments, tasks = pickle.load(f)

In [2]:
for row in df.itertuples():
    if row[1] == 'M': #Main Effect Analysis
        s_index = compile_studies(df, row[0], experiments, tasks)
        if len(s_index) >= 12: 
            print('{} : {} experiments; average of {:.2f} subjects per experiment'.format(row[2], len(s_index), experiments.iloc[s_index].Subjects.mean()))
            
            
        else:
            print("{} : only {} experiments - not analyzed!".format(category, len(s_index)))
            
    if row[1] == 'C': # Contrast Analysis
        study1 = df.loc[row[0],1]
        s1 = compile_studies(df, row[0], experiments, tasks)
        experiments1 = experiments.loc[s1].reset_index(drop=True)

        study2 = df.loc[row[0]+1,1]
        s2 = compile_studies(df, row[0]+1, experiments, tasks)
        experiments2 = experiments.loc[s2].reset_index(drop=True)
        
        if len(s1) >= 12 and len(s2) >= 12:
            if not isfile("{}/ALE/Results/{}_cFWE05.nii".format(cwd, study1)):
                compute_ale(s1, experiments1, study1, noise_repeat=5000)
                contribution(s1, experiments1, tasks, study1)
            if not isfile("{}/ALE/Results/{}_cFWE05.nii".format(cwd, study2)):
                compute_ale(s2, experiments2, study2, noise_repeat=5000)
                contribution(s2, experiments2, tasks, study2)
                
            for i in reversed(s1):
                if i in s2:
                    s1.remove(i)
                    s2.remove(i)
                    
            experiments1 = experiments.loc[s1].reset_index(drop=True)
            experiments2 = experiments.loc[s2].reset_index(drop=True)
            compute_contrast(experiments1, study1, experiments2, study2)
                
            

Other : 19 experiments; average of 22.26 subjects per experiment
Cwclassic : 37 experiments; average of 25.76 subjects per experiment
Cwvariant : 21 experiments; average of 26.33 subjects per experiment
Loading contrast.
Loading conjunction.
Computing positive contrast.
Randomising (positive).
Computing negative contrast.
Randomising (negative).
Inference and printing.
Computing conjunction.
Computing positive contrast.
Randomising (positive).
Computing negative contrast.
Randomising (negative).
Inference and printing.
Computing conjunction.
CwIvsC : 58 experiments; average of 25.97 subjects per experiment
CwIvsN : 41 experiments; average of 22.22 subjects per experiment
CwIvsNwords : 17 experiments; average of 22.24 subjects per experiment
CwIvsNsymbolsletters : 23 experiments; average of 22.30 subjects per experiment
CwIvsC - illustrate Foci
CwIvsC - computing ALE
CwIvsC - permutation-null PDF
CwIvsC - computing p-values
CwIvsC - simulating noise


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    7.6s


KeyboardInterrupt: 

#### Contrast